In [14]:
import pandas as pd
import hopsworks
import os
import requests

In [15]:
player_stats_gw_url = "https://fantasy.premierleague.com/api/event/{gw}/live/"
bootstrap_url = "https://fantasy.premierleague.com/api/bootstrap-static/"

In [16]:
try:
    with open('./hopsworks/hopsworks-api-key.txt', 'r') as file:
        os.environ["HOPSWORKS_API_KEY"] = file.read().rstrip()
except:
    print("In production mode")
    
project = hopsworks.login()
print(project.description)

2024-12-28 10:27:42,638 INFO: Initializing external client
2024-12-28 10:27:42,640 INFO: Base URL: https://c.app.hopsworks.ai:443
2024-12-28 10:27:43,954 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1159321
Default project


In [17]:
response = requests.get(bootstrap_url).json()

# Extract the current and next gameweek statuses
current_gameweek = next(
    (gw for gw in response['events'] if gw['is_current']), None
)

print(current_gameweek["id"])

18


In [18]:
with open ('./state/prev_gameweek.txt', 'r') as file:
    prev_gameweek = file.readline()
print(prev_gameweek)

18


In [21]:
if prev_gameweek != current_gameweek["id"] and current_gameweek["finished"]:
    player_stats_gw_url_formatted = player_stats_gw_url.format(gw=current_gameweek["id"])
    player_stats = requests.get(player_stats_gw_url_formatted).json()


    player_stats_gw_df = pd.DataFrame(player_stats["elements"]).drop(columns=["explain", "modified"])
    player_stats_gw_df = pd.concat(
        [player_stats_gw_df.drop(columns=["stats"]), player_stats_gw_df["stats"].apply(pd.Series)], 
        axis=1
    )

    player_stats_gw_df["gameweek"] = current_gameweek
    print("Saving player stats for gameweek {}".format(current_gameweek["id"]))

Saving player stats for gameweek 18


In [20]:
player_stats_gw_df.head()

,id,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,...,threat,ict_index,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,total_points,in_dreamteam,gameweek
0,1,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0,0.00,0.00,0.00,0.00,0,False,NaN
1,2,71,0,0,1,0,0,0,0,0,...,2.0,2.7,1,0.00,0.10,0.10,0.02,2,False,NaN
2,3,90,0,0,1,0,0,0,0,0,...,11.0,5.0,1,0.27,0.05,0.32,0.16,6,False,NaN
3,4,90,1,0,1,0,0,0,0,1,...,62.0,12.5,1,1.08,0.08,1.16,0.16,7,False,NaN
4,5,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0,0.00,0.00,0.00,0.00,0,False,NaN
